# `Промышленное машинное обучение на Spark`
## `Занятие 04: Детали SQL и Spark`

О чём можно узнать из этого ноутбука:

* RDD API
* Pivot/Unpivot
* Window function
* UDF
* Формат Parquet

Для эффективной работы UDF необходимо установить библиотеку, реализующую передачу данных между Spark и Python в Arrow формате:

In [1]:
! pip3 install pyspark pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

conf = (
    SparkConf()
        .set('spark.ui.port', '4050')
        .setMaster('local[*]')
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

23/10/27 22:40:16 WARN Utils: Your hostname, vm-01 resolves to a loopback address: 127.0.1.1; using 10.128.0.16 instead (on interface eth0)
23/10/27 22:40:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/27 22:40:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/27 22:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


### `Spark RDD API`

Помимо рассмотренных ранне API (DataFrame API & SQL AP) работы с данными в спарк, существует ещё один интерфейс - RDD API. В нём производится работа с RDD напрямую.

### Напоминание

<div style='font-size:1.2em; background-color:#90e0ef; border-radius: 40px; padding:2em; margin: 0 10px 20px'> 
    <b>RDD </b>(resilent distrubuted dataset) - это фундаментальная структура данных Spark, которая представляет собой неизменяемый набор данных, который вычисляются и располагается на разных узлах кластера.
</div>

![rdd_repetition](images/rdd_repetition.png)

* [Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
* [Документация](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)

Для того, чтобы создать RDD необходимо к некоторой коллекции объектов применимеить операциюю parallelize. В результате работы spark разобъёт данные на куски (партиции) и отправит её части на разные worker ноды.

In [3]:
data = [
    (1, 10),
    (2, 41),
    (0, 12),
    (2, 64),
    (2, 22),
    (1, 11),
    (0, 94),
]
dist_data = sc.parallelize(data)
dist_data

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

Обратной операцией к операции parallelize является метод collect, который наоборот создаёт коллекцию из данных, хранящихся на различных worker нодах.

In [4]:
dist_data.collect()

[(1, 10), (2, 41), (0, 12), (2, 64), (2, 22), (1, 11), (0, 94)]

![parallelize-collect](images/parallelize-collect.png)

In [5]:
# К элементам RDD также как и к SparkDataFrame применять различные операции
dist_data.map(lambda x: (x[0] + x[1], x[0])).collect()

[(11, 1), (43, 2), (12, 0), (66, 2), (24, 2), (12, 1), (94, 0)]

Также можно перевести RDD в Spark.DataFrame и обратно

In [6]:
# RDD -> Spark DataFrame 
dist_df = dist_data.toDF(['key', 'value'])

# Spark DataFrame -> RDD 
dist_rdd = dist_df.rdd
dist_df, dist_rdd

(DataFrame[key: bigint, value: bigint],
 MapPartitionsRDD[10] at javaToPython at NativeMethodAccessorImpl.java:0)

Применим операцию групировки по ключу с применением оперции агрегирующей оперцией суммы к значениям, которые имеют один и тот же ключ

In [7]:
dist_data.groupByKey().mapValues(sum).collect()

[(2, 127), (0, 106), (1, 21)]

RDD API поддердивает прямую загрузку из текстового файла. При этом каждая строка будет интерпретироваться, как отдельный элемент RDD

In [8]:
! echo "Hello, sample RDD" > text.txt
! echo "This RDD contains three lines" >> text.txt
! echo "This is the last line" >> text.txt
! echo "" >> text.txt
! echo "Just kidding, it contains five lines" >> text.txt

In [9]:
text_data = sc.textFile('text.txt')
text_data, text_data.collect()

(text.txt MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0,
 ['Hello, sample RDD',
  'This RDD contains three lines',
  'This is the last line',
  '',
  'Just kidding, it contains five lines'])

Теперь к данному RDD можно применять стандартные Spark операции 

In [10]:
distinct_words = (
    text_data
        .filter(lambda x: len(x)) # отбираем только не пустые строки
        .flatMap(lambda x: x.split(' ')) # разбиваем все строки на слова и переводим список
        .distinct() # берём только уникальные слова
)
# Будьте внимательны, если такой файл существует,
# то spark будет выдавать ошибку
distinct_words.saveAsTextFile('words.txt')

# можно вывести отладочную информацию по данному RDD
print(distinct_words.toDebugString().decode())

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[21] at mapPartitions at PythonRDD.scala:160 []
 |  ShuffledRDD[20] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[19] at distinct at /tmp/ipykernel_2175562/1777586505.py:5 []
    |  PythonRDD[18] at distinct at /tmp/ipykernel_2175562/1777586505.py:5 []
    |  text.txt MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0 []
    |  text.txt HadoopRDD[16] at textFile at NativeMethodAccessorImpl.java:0 []


Снизу вверх показаны все низкоуровневые операции (lineage), которые были применены к данному RDD c самого начала его создания 

Для того, чтобы переиспользовать посчитанные значения в рамках текущей сессии стоит использовать метод `.cache`, который сохраняет результат вычислений вершины графа вычислений в оперативной памяти. Это нужно для того, чтобы оперции, работающие поверх данной получали результат операций из оперативной памяти, а не считывались с диска.

Метод `.persist` позволяет сохранять промежуточные вычисления в рамках текущей сессии с более тонкой настройкой места хранения (жёсткий диск, оперативная память, ...)

In [11]:
distinct_words_cached = distinct_words.cache()
print(distinct_words_cached.toDebugString().decode())

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |  MapPartitionsRDD[21] at mapPartitions at PythonRDD.scala:160 [Memory Serialized 1x Replicated]
 |  ShuffledRDD[20] at partitionBy at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
 +-(2) PairwiseRDD[19] at distinct at /tmp/ipykernel_2175562/1777586505.py:5 [Memory Serialized 1x Replicated]
    |  PythonRDD[18] at distinct at /tmp/ipykernel_2175562/1777586505.py:5 [Memory Serialized 1x Replicated]
    |  text.txt MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
    |  text.txt HadoopRDD[16] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]


Как можно заметить после операции cache появились дополнительные вершины в графе вычислений, в которых указаны место расположения данных и количество их реплик: Memory Serialized 1x Replicated

In [12]:
distinct_words_cached.collect()
print(distinct_words_cached.toDebugString().decode())

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |       CachedPartitions: 2; MemorySize: 298.0 B; DiskSize: 0.0 B
 |  MapPartitionsRDD[21] at mapPartitions at PythonRDD.scala:160 [Memory Serialized 1x Replicated]
 |  ShuffledRDD[20] at partitionBy at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
 +-(2) PairwiseRDD[19] at distinct at /tmp/ipykernel_2175562/1777586505.py:5 [Memory Serialized 1x Replicated]
    |  PythonRDD[18] at distinct at /tmp/ipykernel_2175562/1777586505.py:5 [Memory Serialized 1x Replicated]
    |  text.txt MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
    |  text.txt HadoopRDD[16] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]


Для сохранения данных между сессиями можно использовать `.checkpoint`. Особенность этого метода — изменение графа вычислений.
Цепочка вычислений для сохраняемого RDD будет удалена.

Сокращение цепочки вычислений полезно в случае больших графов, например, в итеративных алгоритмах.

In [13]:
distinct_first_words = (
    text_data
        .filter(lambda x: len(x))
        .flatMap(lambda x: x.split(' ')[0])
        .distinct()
)

sc.setCheckpointDir('./checkpoints')

distinct_first_words.checkpoint()
print(distinct_first_words.toDebugString().decode())

(2) PythonRDD[30] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[29] at mapPartitions at PythonRDD.scala:160 []
 |  ShuffledRDD[28] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[27] at distinct at /tmp/ipykernel_2175562/2415635507.py:5 []
    |  PythonRDD[26] at distinct at /tmp/ipykernel_2175562/2415635507.py:5 []
    |  text.txt MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0 []
    |  text.txt HadoopRDD[16] at textFile at NativeMethodAccessorImpl.java:0 []


In [14]:
distinct_first_words.collect()
print(distinct_first_words.toDebugString().decode())

(2) PythonRDD[30] at RDD at PythonRDD.scala:53 []
 |  ReliableCheckpointRDD[31] at collect at /tmp/ipykernel_2175562/656455338.py:1 []


Как можно увидеть предыдущий граф вычилений был полность удалён, и теперь вычисления начинаются с загрузки контрольной точки: ReliableCheckpointRDD

### `Pivot/Unpivot`

Скачаем модельные данные и распакуем их в директорию ml-1m

In [15]:
! wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
! unzip -o ml-1m.zip

--2023-10-27 22:40:37--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  3.91MB/s    in 1.4s    

2023-10-27 22:40:40 (3.91 MB/s) - ‘ml-1m.zip.2’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [16]:
# указываем схему данных для первой таблицы
schema = (
    T.StructType()
        .add('movie_id', T.IntegerType()) # колонка movie_id будет иметь целочиселнный тип
        .add('movie', T.StringType()) # колонка movie будет иметь строковый
        .add('categories', T.StringType()) # и колонка categories также будет строкой
)
# загружаем в spark данные из первой таблицы
movies_df = (
    spark.read
      .format('csv') # указываем формат считываем данных
      .option("header", False) # считываем данные без заголовка
      .option("sep", '::') # в качестве разделителя столбцов указываем ::
      .schema(schema) # в качестве схемы указываем описанную выше схему
      .load('./ml-1m/movies.dat') # указываем источник данных
)

# тоже самое только для второй таблицы
schema = (
    T.StructType()
        .add('user_id', T.IntegerType())
        .add('movie_id', T.IntegerType())
        .add('rating', T.FloatType()) # данный тип - вещественные числа
        .add('timestamp', T.StringType())
)
ratings_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/ratings.dat')
)

# тоже самое только для третьей таблицы
schema = (
    T.StructType()
        .add('user_id', T.IntegerType())
        .add('gender', T.StringType())
        .add('age', T.IntegerType())
        .add('occupation', T.IntegerType())
        .add('zip-code', T.StringType())
)
users_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/users.dat')
)

Во многих задачах зачастую встречается необходимость сформировать колонки на основе данных, хранящихся в другом столбце. То есть требуется перевести таблицу в wide формат.

Например, данная операция нужна была во второй лекции, когда для значения класса нужно было получить one-hot представление

##### `Визуализация того, как это устроено в pandas`

![pandas-pivot](images/pandas-dataframe-pivot-1.svg)

Spark также поддерживает данную операцию, но делается это вызовом не одного метода, а нескольких последовательных:

groupBy -> pivot -> agg

In [17]:
ratings_df.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|    1193|   5.0|978300760|
|      1|     661|   3.0|978302109|
|      1|     914|   3.0|978301968|
|      1|    3408|   4.0|978300275|
|      1|    2355|   5.0|978824291|
|      1|    1197|   3.0|978302268|
|      1|    1287|   5.0|978302039|
|      1|    2804|   5.0|978300719|
|      1|     594|   4.0|978302268|
|      1|     919|   4.0|978301368|
|      1|     595|   5.0|978824268|
|      1|     938|   4.0|978301752|
|      1|    2398|   4.0|978302281|
|      1|    2918|   4.0|978302124|
|      1|    1035|   5.0|978301753|
|      1|    2791|   4.0|978302188|
|      1|    2687|   3.0|978824268|
|      1|    2018|   4.0|978301777|
|      1|    3105|   5.0|978301713|
|      1|    2797|   4.0|978302039|
+-------+--------+------+---------+
only showing top 20 rows



In [18]:
pivot_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id')
        .agg(F.first(ratings_df.rating))
)
pivot_df.where(pivot_df.user_id < 4).toPandas()

23/10/27 22:40:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/10/27 22:40:59 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB                                                                                    (0 + 1) / 1]


,user_id,1,2,3,4,5,6,7,8,9,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
top_movies_df = ratings_df.groupBy(
    ratings_df.movie_id # групируем по movie_id
).agg(
    F.count(ratings_df.rating).alias('rates'), # создаём колонку rates c количество оценок
    F.mean(ratings_df.rating).alias('avg_rating') # создаём колонку avg_rating с средней оценкой
).sort(
    'rates', ascending=False # упорядочиваем фильмы по количеству оценок в убывающем порядке
).limit(100) # берём только 100 самых оцениваемых фильмов

top_movies_df.show()

+--------+-----+------------------+
|movie_id|rates|        avg_rating|
+--------+-----+------------------+
|    2858| 3428|4.3173862310385065|
|     260| 2991| 4.453694416583082|
|    1196| 2990| 4.292976588628763|
|    1210| 2883| 4.022892819979188|
|     480| 2672|3.7638473053892216|
|    2028| 2653| 4.337353938937053|
|     589| 2649| 4.058512646281616|
|    2571| 2590| 4.315830115830116|
|    1270| 2583|3.9903213317847466|
|     593| 2578|4.3518231186966645|
|    1580| 2538| 3.739952718676123|
|    1198| 2514| 4.477724741447892|
|     608| 2513| 4.254675686430561|
|    2762| 2459| 4.406262708418057|
|     110| 2443| 4.234957020057307|
|    2396| 2369| 4.127479949345715|
|    1197| 2318|4.3037100949094045|
|     527| 2304| 4.510416666666667|
|    1617| 2288| 4.219405594405594|
|    1265| 2278| 3.953028972783143|
+--------+-----+------------------+
only showing top 20 rows



In [20]:
top_movies = top_movies_df.rdd.map(lambda x: x.movie_id).collect()

In [21]:
pivot_top_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id', top_movies)
        .agg(F.first(ratings_df.rating))
)
pivot_top_df.where(pivot_df.user_id < 3).toPandas()

,user_id,2858,260,1196,1210,480,2028,589,2571,1270,...,750,2699,39,21,1393,2804,588,2406,1220,733
0,1,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN
1,2,4.0,NaN,5.0,4.0,5.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


В качестве второго аргумента в методе pivot можно указать подмножество допустимых значений на основе, которых сформируются новые столбцы

In [22]:
# формируем таблицу с столбцы только по top-100 самых оцениваемых фильмов
pivot_top_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id', top_movies) # вторым параметром ограничиваем дотупстимые столбцы
        .agg(F.first(ratings_df.rating))
)
pivot_top_df.where(pivot_top_df.user_id < 4).toPandas()

,user_id,2858,260,1196,1210,480,2028,589,2571,1270,...,750,2699,39,21,1393,2804,588,2406,1220,733
0,1,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN
1,3,4.0,5.0,4.0,4.0,4.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,2,4.0,NaN,5.0,4.0,5.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


Как можно заметить кол-во пустых столбцов стало гораздо меньше

Скачаем данные из предыдущей лекции, с которыми будем в дальнейшем работать

In [23]:
import json
import requests
import subprocess
import zipfile

folder_url = 'https://disk.yandex.lt/d/JnDy1h48pJI7IA'
file_url = '/m5-forecating-accuracy.zip'
# запрос ссылки на скачивание
response = requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download',
                 params={'public_key': folder_url, 'path': file_url}) 
# 'парсинг' ссылки на скачивание
data_link = response.json()['href'] 	

filename = 'm5-forecating-accuracy.zip'
path = "./m5-forecasting-accuracy"

# запускаем скачивание вызовом команды wget из python
subprocess.run(
    ['wget', '-O', filename, data_link], # команда для исполнения
    stdout=subprocess.DEVNULL, # убираем печать отладочной информации
    stderr=subprocess.STDOUT
)

#распакуем данные по пути path
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [24]:
# считаем данные
df_validation = (
    spark.read.format('csv')
      .option("inferSchema", True) # указываем spark самому определить тип данных
      .option("header", True)
      .option("sep", ',')
      .load(f"{path}/sales_train_validation.csv")
)
df_validation.limit(10).toPandas()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


Для операции pivot существует противоположная операция "вытягивания" таблицы в длину на основе значений из столбцов.

Для этого применяется оперция stack:
```SQL
stack(
    N, new_value_name_1, old_colname_1, ..., new_value_name_N, old_colname_N
) as (new_colname, value_colname)
```

In [25]:
unpivot_expression = "stack(2, 'd_1_new', d_1, 'd_2', d_2) as (d, sales)"
unpivot_df = (
    df_validation
        .select('id', F.expr(unpivot_expression)) 
)

unpivot_df.show(truncate=False)

+-----------------------------+-------+-----+
|id                           |d      |sales|
+-----------------------------+-------+-----+
|HOBBIES_1_001_CA_1_validation|d_1_new|0    |
|HOBBIES_1_001_CA_1_validation|d_2    |0    |
|HOBBIES_1_002_CA_1_validation|d_1_new|0    |
|HOBBIES_1_002_CA_1_validation|d_2    |0    |
|HOBBIES_1_003_CA_1_validation|d_1_new|0    |
|HOBBIES_1_003_CA_1_validation|d_2    |0    |
|HOBBIES_1_004_CA_1_validation|d_1_new|0    |
|HOBBIES_1_004_CA_1_validation|d_2    |0    |
|HOBBIES_1_005_CA_1_validation|d_1_new|0    |
|HOBBIES_1_005_CA_1_validation|d_2    |0    |
|HOBBIES_1_006_CA_1_validation|d_1_new|0    |
|HOBBIES_1_006_CA_1_validation|d_2    |0    |
|HOBBIES_1_007_CA_1_validation|d_1_new|0    |
|HOBBIES_1_007_CA_1_validation|d_2    |0    |
|HOBBIES_1_008_CA_1_validation|d_1_new|12   |
|HOBBIES_1_008_CA_1_validation|d_2    |15   |
|HOBBIES_1_009_CA_1_validation|d_1_new|2    |
|HOBBIES_1_009_CA_1_validation|d_2    |0    |
|HOBBIES_1_010_CA_1_validation|d_1

Теперь произвём обратную оперцию "расширения" (pivot) для новой таблицы

In [26]:
(
    unpivot_df
        .groupBy(unpivot_df.id)
        .pivot('d')
        .agg(F.sum(unpivot_df.sales))
).show(truncate=False)

+-------------------------------+-------+---+
|id                             |d_1_new|d_2|
+-------------------------------+-------+---+
|HOBBIES_1_304_CA_4_validation  |0      |0  |
|FOODS_3_257_CA_3_validation    |1      |0  |
|HOUSEHOLD_1_157_CA_2_validation|0      |0  |
|FOODS_2_387_CA_1_validation    |0      |0  |
|FOODS_2_225_CA_3_validation    |4      |6  |
|FOODS_2_375_CA_2_validation    |0      |0  |
|FOODS_2_354_CA_4_validation    |0      |0  |
|FOODS_3_297_TX_1_validation    |0      |0  |
|HOUSEHOLD_1_319_CA_3_validation|1      |1  |
|HOUSEHOLD_2_090_CA_4_validation|0      |0  |
|FOODS_3_568_CA_4_validation    |0      |0  |
|HOBBIES_1_364_TX_2_validation  |3      |0  |
|FOODS_1_101_CA_2_validation    |0      |0  |
|FOODS_3_558_TX_1_validation    |1      |2  |
|FOODS_2_011_CA_1_validation    |1      |1  |
|FOODS_3_035_CA_3_validation    |3      |2  |
|FOODS_2_174_CA_2_validation    |0      |0  |
|HOBBIES_1_396_CA_4_validation  |0      |0  |
|HOBBIES_2_044_CA_2_validation  |0

Проверим, что в ходе операций unpivot->pivot данные остались такими же какими они и были изначально, взяв последнюю строку из таблицы выше

In [27]:
(
    df_validation
        .where(df_validation.id == 'HOBBIES_1_320_CA_3_validation')
        .select('id', 'd_1', 'd_2')
).show(truncate=False)

+-----------------------------+---+---+
|id                           |d_1|d_2|
+-----------------------------+---+---+
|HOBBIES_1_320_CA_3_validation|7  |2  |
+-----------------------------+---+---+



### `Window function`

В SQL запросах часто встречаются оконные функции, которые необходимы при различных для аналитических задач с данными: 

 - нарастающие итоги
 - скользящие средние
 - ранжирование


**Оконная функция в SQL** - функция, которая работает с выделенным набором строк (окном, партицией) и выполняет вычисление для этого набора строк в отдельном столбце. 

##### `Сравнение оконной функции с групировкой`

![window_func](images/window_func.png)

В pyspark также есть возможность работы с оконными функциями

In [41]:
# импортируем модуль, в котором располагаются оконные функции
from pyspark.sql.window import Window

In [42]:
data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('Scott', 'Finance', 3300),
    ('Jen', 'Finance', 3900), 
    ('Jeff', 'Marketing', 3000),
    ('Kumar', 'Marketing', 2000),
    ('Saif', 'Sales', 4100)
]
 
columns = ["emp_id", "dept_id", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema(), df.show(truncate=False)

root
 |-- emp_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- salary: long (nullable = true)

+-------+---------+------+
|emp_id |dept_id  |salary|
+-------+---------+------+
|James  |Sales    |3000  |
|Michael|Sales    |4600  |
|Robert |Sales    |4100  |
|Maria  |Finance  |3000  |
|Scott  |Finance  |3300  |
|Jen    |Finance  |3900  |
|Jeff   |Marketing|3000  |
|Kumar  |Marketing|2000  |
|Saif   |Sales    |4100  |
+-------+---------+------+



(None, None)

Cоздадим новую таблицу, в которой будет с агрегирована информация о средней зарплате в пределах департамента и объеденим её с таблицей с информацией о сотрудниках

In [43]:
avg_salaries_df = df.groupBy(
    df.dept_id # по какому полю агрегируем
).agg(
    F.mean(df.salary # функция агрегации
).alias('avg_salary')) # задаём имя новой колнке

# объединяем с таблицей сотрудников
df.join(avg_salaries_df, on='dept_id', how='inner').show()

+---------+-------+------+----------+
|  dept_id| emp_id|salary|avg_salary|
+---------+-------+------+----------+
|    Sales|  James|  3000|    3950.0|
|    Sales|Michael|  4600|    3950.0|
|    Sales| Robert|  4100|    3950.0|
|  Finance|  Maria|  3000|    3400.0|
|    Sales|   Saif|  4100|    3950.0|
|  Finance|  Scott|  3300|    3400.0|
|  Finance|    Jen|  3900|    3400.0|
|Marketing|   Jeff|  3000|    2500.0|
|Marketing|  Kumar|  2000|    2500.0|
+---------+-------+------+----------+



In [44]:
# создаём окно, в котором указываем по какому полю будет происходит создание окон
wspec = Window.partitionBy('dept_id')

(
    df
        .withColumn( # собираем зарплаты сотрудников в пределах своего департамента
            'salaries_list',
            F.collect_list(df.salary).over(wspec)
        ) 
        .withColumn(
            'avg_salary', # считаем среднюю зарплату в пределах департамента
            F.mean(df.salary).over(wspec) 
        )
).show(truncate=False)

+-------+---------+------+------------------------+----------+
|emp_id |dept_id  |salary|salaries_list           |avg_salary|
+-------+---------+------+------------------------+----------+
|Maria  |Finance  |3000  |[3000, 3300, 3900]      |3400.0    |
|Scott  |Finance  |3300  |[3000, 3300, 3900]      |3400.0    |
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3400.0    |
|Jeff   |Marketing|3000  |[3000, 2000]            |2500.0    |
|Kumar  |Marketing|2000  |[3000, 2000]            |2500.0    |
|James  |Sales    |3000  |[3000, 4600, 4100, 4100]|3950.0    |
|Michael|Sales    |4600  |[3000, 4600, 4100, 4100]|3950.0    |
|Robert |Sales    |4100  |[3000, 4600, 4100, 4100]|3950.0    |
|Saif   |Sales    |4100  |[3000, 4600, 4100, 4100]|3950.0    |
+-------+---------+------+------------------------+----------+



Помимо указания полей, по которым будут созданы окна (partitionBy), можно ещё указать по какому полю должны быть упорядочены значения в пределах одного окна (orderBy). При добвалении порядка вычисление значений на основе оконной функции происходит последовательно, а не сразу. 

In [45]:
# создаём окна по полю dep_id и упорядочеваем значения в них на основе поля salary
wspec = Window.partitionBy('dept_id').orderBy('salary') 
(
    df
        # последовательно собираем список зарплат сотрудников в одном департаменте
        .withColumn(
            'salaries_list', 
            F.collect_list(df.salary).over(wspec) 
        )
        # функцией row_number указываем порядок зарплаты сотрудника в пределах своего депратамента
        .withColumn( 
            'row_number', 
            F.row_number().over(wspec)
        )
        # последовательно считаем среднеюю зарплату сотрудников в пределах своего департамента
        .withColumn(
            'avg_salary',
            F.mean(df.salary).over(wspec)
        )
).show(truncate=True)

+-------+---------+------+--------------------+----------+------------------+
| emp_id|  dept_id|salary|       salaries_list|row_number|        avg_salary|
+-------+---------+------+--------------------+----------+------------------+
|  Maria|  Finance|  3000|              [3000]|         1|            3000.0|
|  Scott|  Finance|  3300|        [3000, 3300]|         2|            3150.0|
|    Jen|  Finance|  3900|  [3000, 3300, 3900]|         3|            3400.0|
|  Kumar|Marketing|  2000|              [2000]|         1|            2000.0|
|   Jeff|Marketing|  3000|        [2000, 3000]|         2|            2500.0|
|  James|    Sales|  3000|              [3000]|         1|            3000.0|
| Robert|    Sales|  4100|  [3000, 4100, 4100]|         2|3733.3333333333335|
|   Saif|    Sales|  4100|  [3000, 4100, 4100]|         3|3733.3333333333335|
|Michael|    Sales|  4600|[3000, 4100, 4100...|         4|            3950.0|
+-------+---------+------+--------------------+----------+------

При вычислении оконных функций можно указать границы окна для текущего элемента через специальный дополнительный метод rowsBetween(start_position, end_position).

Например, следуюший вызов:
 - rowsBetween(0, 1) - будет ограниичивать допустимую границу вычисления для текущего элемента следующей строкой
 - rowsBetween(-2, 4) - будет ограничивать вычисления двумя предыдущими элементами и четырьмя последующими

Для обозначения наиболее частоиспользуемых границ используют ключевые слова:
 - Window.currentRow - текущая позиция
 - Window.unboundedFollowing - все последующие значения
 - Window.unboundedPreceding - все предыдущие значения


![rowsWindow](images/rowsWindow.png)

In [46]:
# создадим окно, в котором в вычислениях затрагиваются только 
# все последующие элементы окна + текущий
wspec = (
    Window
        .partitionBy('dept_id')
        .orderBy('salary')
        .rowsBetween(Window.currentRow, Window.unboundedFollowing)
)
(
    df
        .withColumn(
            'salaries_list', # собираем список зарплат последующих сотрудников + текущий
            F.collect_list(df.salary).over(wspec)
        )
        .withColumn(
            'avg_salary', # cчитаем среднюю зарплату для следующих сотрудников + текущий
            F.mean(df.salary).over(wspec)
        )
).show()

+-------+---------+------+--------------------+-----------------+
| emp_id|  dept_id|salary|       salaries_list|       avg_salary|
+-------+---------+------+--------------------+-----------------+
|  Maria|  Finance|  3000|  [3000, 3300, 3900]|           3400.0|
|  Scott|  Finance|  3300|        [3300, 3900]|           3600.0|
|    Jen|  Finance|  3900|              [3900]|           3900.0|
|  Kumar|Marketing|  2000|        [2000, 3000]|           2500.0|
|   Jeff|Marketing|  3000|              [3000]|           3000.0|
|  James|    Sales|  3000|[3000, 4100, 4100...|           3950.0|
| Robert|    Sales|  4100|  [4100, 4100, 4600]|4266.666666666667|
|   Saif|    Sales|  4100|        [4100, 4600]|           4350.0|
|Michael|    Sales|  4600|              [4600]|           4600.0|
+-------+---------+------+--------------------+-----------------+



Помимо того, что можно указывать границы окна на основе количества последующих и предыдущих элементво в упорядоченном списке, можно указать и границы на основе значения поля по которому происходит упорядочивание. 

Например, выражение **orderBy(colname).rangeBetween(-400, 400)** для текущего элемента окна с со значением **X** в столбце colname возьмёт только те оконные элементы, у которых значения y в столбце colname удовлетворяет следующему неравенству: 

X - 400 <= y <= X + 400

In [47]:
# считаем значения оконные выражения только для тех сотрудников из одного депратамента,
# у которых различие в зарплатах не превосходит 400 у.е.

wspec = Window.partitionBy('dept_id').orderBy('salary').rangeBetween(-400, 400)
(
    df
        .withColumn( # собираем значения зарплат сотрудников, попавших в интервал [X-400, X+400]
            'salaries_list', 
            F.collect_list(df.salary).over(wspec)
        )
        .withColumn( # среднюю зарплату сотрудников, попавших в интервал [X-400, X+400]
            'avg_salary', 
            F.mean(df.salary).over(wspec))
).show()

+-------+---------+------+-------------+----------+
| emp_id|  dept_id|salary|salaries_list|avg_salary|
+-------+---------+------+-------------+----------+
|  Maria|  Finance|  3000| [3000, 3300]|    3150.0|
|  Scott|  Finance|  3300| [3000, 3300]|    3150.0|
|    Jen|  Finance|  3900|       [3900]|    3900.0|
|  Kumar|Marketing|  2000|       [2000]|    2000.0|
|   Jeff|Marketing|  3000|       [3000]|    3000.0|
|  James|    Sales|  3000|       [3000]|    3000.0|
| Robert|    Sales|  4100| [4100, 4100]|    4100.0|
|   Saif|    Sales|  4100| [4100, 4100]|    4100.0|
|Michael|    Sales|  4600|       [4600]|    4600.0|
+-------+---------+------+-------------+----------+



Ещё наиболее частотные оконные функции:
 - cume_dist - доля элементов, в окне меньших или равных данному
 - lag(colname, N) - значение столбца сolname, который отстаёт от данного на N элементов
 - lag(colname, M) - значение столбца сolname, который опережает данный на M элементов

In [48]:
wspec = Window.partitionBy('dept_id').orderBy('salary')
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
        .withColumn('cume_dist', F.cume_dist().over(wspec))
        .withColumn('lag', F.lag(df.salary, 1).over(wspec))
        .withColumn('lead', F.lead(df.salary, 1).over(wspec))
).show(truncate=False) 

+-------+---------+------+------------------------+------------------+------------------+----+----+
|emp_id |dept_id  |salary|salaries_list           |avg_salary        |cume_dist         |lag |lead|
+-------+---------+------+------------------------+------------------+------------------+----+----+
|Maria  |Finance  |3000  |[3000]                  |3000.0            |0.3333333333333333|NULL|3300|
|Scott  |Finance  |3300  |[3000, 3300]            |3150.0            |0.6666666666666666|3000|3900|
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3400.0            |1.0               |3300|NULL|
|Kumar  |Marketing|2000  |[2000]                  |2000.0            |0.5               |NULL|3000|
|Jeff   |Marketing|3000  |[2000, 3000]            |2500.0            |1.0               |2000|NULL|
|James  |Sales    |3000  |[3000]                  |3000.0            |0.25              |NULL|4100|
|Robert |Sales    |4100  |[3000, 4100, 4100]      |3733.3333333333335|0.75              |3000|4100|


Функции для работы с порядком элементов в окне:

 - nth_value - n-ый элментов в текущем окне
 - ntile - количество элементов в текущем окне меньших чем заданный
 - dense_rank, rank - функции ранга элмента.Различия в работе в них проявляются, если в столбце встречаются одинковые значения. Хорошо  написано про это [ здесь ](http://www.sql-tutorial.ru/ru/book_rank_dense_rank_functions.html)

In [49]:
(
    df      
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('nth_value', F.nth_value(df.salary, 2).over(wspec))
        .withColumn('ntile', F.ntile(2).over(wspec))
        .withColumn('dense_rank', F.dense_rank().over(wspec))
        .withColumn('percent_rank', F.percent_rank().over(wspec))
        .withColumn('rank', F.rank().over(wspec))
).show(truncate=True)

+-------+---------+------+--------------------+---------+-----+----------+------------------+----+
| emp_id|  dept_id|salary|       salaries_list|nth_value|ntile|dense_rank|      percent_rank|rank|
+-------+---------+------+--------------------+---------+-----+----------+------------------+----+
|  Maria|  Finance|  3000|              [3000]|     NULL|    1|         1|               0.0|   1|
|  Scott|  Finance|  3300|        [3000, 3300]|     3300|    1|         2|               0.5|   2|
|    Jen|  Finance|  3900|  [3000, 3300, 3900]|     3300|    2|         3|               1.0|   3|
|  Kumar|Marketing|  2000|              [2000]|     NULL|    1|         1|               0.0|   1|
|   Jeff|Marketing|  3000|        [2000, 3000]|     3000|    2|         2|               1.0|   2|
|  James|    Sales|  3000|              [3000]|     NULL|    1|         1|               0.0|   1|
| Robert|    Sales|  4100|  [3000, 4100, 4100]|     4100|    1|         2|0.3333333333333333|   2|
|   Saif| 

### `UDF`

В предыдущих примеров, при работе с элементами таблиц использовались встроенные методы обработки строк spark таблиц из модуля pyspark.sql.functions. Но фрейморк не запрещает определять свои кастомные функции обработки.

In [52]:
import pandas as pd

@F.pandas_udf('double', F.PandasUDFType.SCALAR)
def add_one(v):
    return v + 1

ratings_df.select(add_one(ratings_df.rating)).show()
ratings_df.withColumn(
    'plus_one', add_one(ratings_df.rating)
).show()

/home/evgeniy/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


+---------------+
|add_one(rating)|
+---------------+
|            6.0|
|            4.0|
|            4.0|
|            5.0|
|            6.0|
|            4.0|
|            6.0|
|            6.0|
|            5.0|
|            5.0|
|            6.0|
|            5.0|
|            5.0|
|            5.0|
|            6.0|
|            5.0|
|            4.0|
|            5.0|
|            6.0|
|            5.0|
+---------------+
only showing top 20 rows

+-------+--------+------+---------+--------+
|user_id|movie_id|rating|timestamp|plus_one|
+-------+--------+------+---------+--------+
|      1|    1193|   5.0|978300760|     6.0|
|      1|     661|   3.0|978302109|     4.0|
|      1|     914|   3.0|978301968|     4.0|
|      1|    3408|   4.0|978300275|     5.0|
|      1|    2355|   5.0|978824291|     6.0|
|      1|    1197|   3.0|978302268|     4.0|
|      1|    1287|   5.0|978302039|     6.0|
|      1|    2804|   5.0|978300719|     6.0|
|      1|     594|   4.0|978302268|     5.0|
| 

/home/evgeniy/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


Также можно определить свои агрегирующие функции

In [37]:
@F.pandas_udf(ratings_df.schema, F.PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def subtract_mean(pdf):
    return pdf.assign(rating=pdf.rating - pdf.rating.mean())

ratings_df.groupby('movie_id').apply(subtract_mean).show()

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------+--------+------------+---------+
|user_id|movie_id|      rating|timestamp|
+-------+--------+------------+---------+
|    181|      31|   0.8865249|977087101|
|    195|      31|-0.113475084|991013952|
|    203|      31|   -2.113475|976929358|
|    223|      31|-0.113475084|976905652|
|    268|      31|   0.8865249|976647137|
|    368|      31|-0.113475084|976670975|
|    517|      31|   0.8865249|976204301|
|    524|      31|   -2.113475|976171096|
|    528|      31|   1.8865249|980039160|
|    531|      31|  -1.1134751|978973034|
|    536|      31|-0.113475084|976137228|
|    543|      31|   0.8865249|976159357|
|    616|      31|-0.113475084|975802599|
|    676|      31|   0.8865249|975684957|
|    678|      31|   0.8865249|989241973|
|    692|      31|-0.113475084|978375055|
|    699|      31|-0.113475084|975563262|
|    710|      31|-0.113475084|978586309|
|    752|      31|   -2.113475|975461295|
|    777|      31|-0.113475084|975520841|
+-------+--------+------------+---

#### `Формат Parquet`

Обработка больших данных увеличивает нагрузку на подсистему хранения — Hadoop хранит данные избыточно для достижения отказоустойчивости. Кроме дисков, нагружаются процессор, сеть, система ввода-вывода и так далее. По мере роста объема данных увеличивается и стоимость их обработки и хранения.

Различные форматы файлов в Hadoop придуманы для решения именно этих проблем. Выбор подходящего формата файла может дать некоторые существенные преимущества:

 - Более быстрое время чтения.
 - Более быстрое время записи.
 - Разделяемые файлы.
 - Поддержка эволюции схем.
 - Расширенная поддержка сжатия.

![parquet](images/data_sizes.png)

Parquet - это столбчатый формат хранения данных. Это помогает повысить производительность,
иногда значительно, разрешая хранение и доступ к данным для каждого столбца&

Столбчатый формат более эффективен, когда вам нужно запросить из таблицы несколько столбцов. Он прочитает только необходимые столбцы, потому что они находятся по соседству. Таким образом, операции ввода-вывода сводятся к минимуму.

![parquet_columns](images/parquet_columns.png)

Сохраним датафрейм ratings_df в формате Parquet и посмотрим его содержимое.

In [64]:
df.write.mode("overwrite").parquet("ratings_df_parquet")

После процедуры сохранения файла можно увидеть два файла в формате parquet. Количество сохранённых файлов равно количеству партиций RDD, которые были у данного датафрейма

In [65]:
# выведем кол-во партиций
df.rdd.getNumPartitions()

2

Sprk позволяет при записи создавать разбиение данных на основе значений некоторого столбца. Таким образом каждое уникальное значение в нём создаст свою партцию.

Зачастую это нужно для эффективной работы с нужной порцией данных, например, когда нужно обработать данные в одной партииции.

In [68]:
# сохраним данные и разобъём их на партции по полю salary
df.write.partitionBy("salary").mode("overwrite").parquet("employees")

Теперь если зайти в директорию employees, то можно увидедеть, что spark создал подпапки с названиями пратиций. Каждая папка хранит данные только своей партиции